In [1]:
import tensorflow as tf

In [47]:
file_queue = tf.train.string_input_producer(
['wine.csv'], shuffle=False, name='file_queue')#tf.data로 대체 방법 확인
#ex) ['wine1.scv', 'wine2.csv', ... ] 이렇게 여러개 불러올 수 있당

In [48]:
reader = tf.TextLineReader()
key, value = reader.read(file_queue)

In [49]:
record_defaults = [[0.], [0.], [0.], [0.], [0.], [0.], [0.], [0.], [0.], [0.], [0.], [0.]]
data = tf.decode_csv(value, record_defaults=record_defaults)

In [50]:
x_batch, y_batch = tf.train.batch([data[0:-1],data[-1:]], batch_size=10)#뒤에 data[-1:]에서 :꼭 붙여줘야한다
#min_after_dequeue = 10000 <- 얼마나 넓은 범위에서 랜덤으로 추출하는지
#capacity = min_after_dequeue + 3 * batch_size <- min_after_dequeue보다 커야하며 뭘 결정하는지는 모르겟다;
#추천 사이즈에서 3은 num_threads + a small safety margin이라는데 적절히 정하면 될거같다
#x_batch, y_batch = tf.train.shuffle_batch([data[0:-1],data[-1:], batch_size = size, capacity = capacity, min_after_dequeue = min_after_dequeue)
#요래 쓰면 스까서 데이터를 뽑아낼 것이다. 

In [51]:
x = tf.placeholder(tf.float32, shape=[None, 11])
y = tf.placeholder(tf.float32, shape=[None, 1])
w = tf.Variable(tf.random_normal([11,1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [52]:
hypothesis = tf.matmul(x,w) + b
mse = tf.reduce_mean(tf.square(hypothesis - y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(mse)

In [53]:
sess = tf.Session()

In [54]:
sess.run(tf.global_variables_initializer())

In [55]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess = sess, coord = coord)#tf.data모듈로 대체하는거 여기도 해당

In [57]:
for step in range(2001):
    x_data, y_data = sess.run([x_batch, y_batch]) 
    mse_val, hy_val, _ = sess.run(
    [mse, hypothesis, train],
    feed_dict={x:x_data, y:y_data})
    if step < 20:
        print(step," step, mse : ",mse_val,"\n y_data : ",y_data, "\nhypo : ",hy_val)


0  step, mse :  20.925 
 y_data :  [[6.]
 [6.]
 [7.]
 [6.]
 [6.]
 [6.]
 [7.]
 [6.]
 [6.]
 [5.]] 
hypo :  [[ 3.1346824]
 [ 6.7195644]
 [ 7.1403103]
 [12.213862 ]
 [12.158193 ]
 [12.158193 ]
 [ 6.9259033]
 [12.213862 ]
 [12.158193 ]
 [ 8.082665 ]]
1  step, mse :  19.544067 
 y_data :  [[5.]
 [5.]
 [5.]
 [5.]
 [7.]
 [4.]
 [8.]
 [7.]
 [5.]
 [8.]] 
hypo :  [[ 0.4189124 ]
 [ 5.057458  ]
 [ 5.1278625 ]
 [ 1.056986  ]
 [10.53257   ]
 [ 5.7974706 ]
 [ 0.5175824 ]
 [ 0.31634736]
 [11.340154  ]
 [ 9.523816  ]]
2  step, mse :  24.865852 
 y_data :  [[7.]
 [5.]
 [7.]
 [6.]
 [8.]
 [6.]
 [6.]
 [3.]
 [5.]
 [6.]] 
hypo :  [[10.299909 ]
 [ 2.7341256]
 [ 2.3756213]
 [ 3.2933602]
 [ 4.844892 ]
 [ 9.435447 ]
 [-2.1051426]
 [-4.205381 ]
 [ 6.957077 ]
 [-1.7926997]]
3  step, mse :  17.2942 
 y_data :  [[6.]
 [7.]
 [5.]
 [5.]
 [7.]
 [7.]
 [7.]
 [6.]
 [7.]
 [5.]] 
hypo :  [[-1.2704325 ]
 [ 4.823667  ]
 [ 5.0438437 ]
 [ 5.231437  ]
 [ 7.468266  ]
 [ 3.9426997 ]
 [ 4.823667  ]
 [-1.2066584 ]
 [ 1.741436  ]
 [ 0.

In [58]:
#batch_size가 한step당 트레이닝 시킬 데이터 양을 결정한다. 데이터 뭉치는 위에 입력 순서대로
#선형으로 이어져 있고 매 학습 step 마다 batch_size만큼 가져와서 학습한다.
#step for문이 종결된 후 다시 실행하면 가장 최근에 학습이 끝난 부분 이후부터 batch_size만큼
#가져와서 다시 학습을 시작한다.
#위 처럼 순서대로 데이터를 가져오는게 아니라 섞고 싶으면 위에 Shuffle여부를 바꿔주면 된느듯

In [59]:
coord.request_stop()
coord.join(threads)

In [ ]:
#약간 c에서 free 같은 느낌으로? 위에 선언했던것을 끝내주는 코드